In [1]:
!pip install transformers datasets evaluate

In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

In [3]:
import pandas as pd
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from transformers import BertForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
dataset = load_dataset('mohammadhossein/SemEvalTask8_SubTaskAMono')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/119757 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [22]:
id2label = {0: "Human", 1: "Bloomz"}
label2id = {"Human": 0, "Bloomz": 1}
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenizer_function(example):
    return tokenizer(example['text'], truncation=True)

ds = dataset.map(tokenizer_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest')

Map:   0%|          | 0/119757 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
ds = ds.remove_columns(['text', 'source', 'id', 'model'])
ds = ds.rename_column('label', 'labels')

In [ ]:
temp = ds['train'].shuffle().shard(num_shards=10, index=0)

temp['labels']

In [11]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    results = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": results["accuracy"]}

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [28]:
training_args = TrainingArguments(
    output_dir="mohammadhossein/SemEvalTask8_SubTaskAMono",
    learning_rate=1e-4,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=64,
    fp16=True,
    gradient_checkpointing=True,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)
model = model.to(device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'].shuffle().shard(num_shards=10, index=0),
    eval_dataset=ds['dev'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.821073,0.663400
2,No log,0.646496,0.773800
3,No log,0.976718,0.733400
4,No log,1.310212,0.730800
5,No log,1.520171,0.732400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/

TrainOutput(global_step=235, training_loss=0.13572782962880237, metrics={'train_runtime': 3229.1261, 'train_samples_per_second': 18.544, 'train_steps_per_second': 0.073, 'total_flos': 1.575417732848352e+16, 'train_loss': 0.13572782962880237, 'epoch': 5.0})